In [1]:
import os, sys, pathlib, json
import pandas as pd

Change labels into human-readable format

In [8]:
class_name_file = pd.read_csv('dataset/id_label_data.csv')

folder_dir = 'dataset/ILSVRC2012_img_val'

val_file_dir = pathlib.Path(folder_dir)
print(val_file_dir.exists())

type(class_name_file['origin'][0])

class_name_file.head()

True


,origin,update,score
0,n02640242,sturgeon,0.310153
1,n02536864,coho,0.158441
2,n01440764,tench,0.127279
3,n01695060,Komodo_dragon,0.122669
4,n02526121,eel,0.116618


In [48]:
for idx in range(len(class_name_file)):
  origin = class_name_file['origin'][idx]
  update = class_name_file['update'][idx]
  origin_path = pathlib.Path(folder_dir, origin)
  update_path = pathlib.Path(folder_dir, update)
  if update_path.exists():
    str_idx = str(idx)
    update_path = pathlib.Path(folder_dir, update+"_"+str_idx)
  os.rename(origin_path, update_path)

Saving ace results

In [2]:
name = 'airfield'
#ace_dir = 'Ace-results2/ACE_Inception_'+name
ace_dir = 'Ace-results2/ACE_DenseNet_'+name
#ace_dir = 'Ace-results2/ACE_Resnet_'+name
#ace_dir = 'Ace-results2/ACE_VGG_'+name

ace_file_dir = pathlib.Path(ace_dir)
print(ace_file_dir.exists())

target_class = 'airfield'

#model = 'Inception'
model = 'DenseNet'
#model= 'Resnet'
#model = 'VGG'

with open(ace_dir+"/"+target_class+"_"+model+"_tcav_scores.json") as f:
  tcav_data = json.load(f)

False


FileNotFoundError: [Errno 2] No such file or directory: 'Ace-results2/ACE_DenseNet_airfield/airfield_DenseNet_tcav_scores.json'

In [221]:
class_names = [name for name in os.listdir(ace_dir) if os.path.isdir(os.path.join(ace_dir, name))]
print(class_names)

concepts_dir = pathlib.Path(ace_dir, 'concepts')
masked_concepts_dir = pathlib.Path(ace_dir, 'masked_results')
results_dir = pathlib.Path(ace_dir, 'results')
summary_dir = pathlib.Path(ace_dir, 'results_summaries')
full_masked_dir = pathlib.Path(masked_concepts_dir, 'full_tcav_masked_results')

ace_data = dict()
ace_data['concept'] = dict()
ace_data['masked'] = dict()
ace_data['full_masked'] = dict()
ace_data['results'] = dict()
ace_data['summary'] = dict()

concept_names = [name for name in os.listdir(concepts_dir) if os.path.isdir(os.path.join(concepts_dir, name))]

for i in range(len(concept_names)):
  name = concept_names[i]
  if 'patches' not in name:
    continue
  each_file_dir = pathlib.Path(concepts_dir, name)
  file_lists = list(each_file_dir.glob("*.png"))
  files = []
  for file in file_lists:
    files.append(str(file))
  ace_data['concept'][name] = dict()
  ace_data['concept'][name]['file'] = files 


['acts', 'cam_results', 'cavs', 'concepts', 'masked_results', 'results', 'results_summaries']


In [222]:
# grad-cam saliency map result (folder name cam_results)
cam_result_dir = pathlib.Path(ace_dir, 'cam_results')
cam_result_conv_dir = pathlib.Path(ace_dir, 'cam_results2')

ace_data['cam_results'] = dict()
ace_data['cam_results_conv'] = dict()

cam_files = list(cam_result_dir.glob("*.png"))
for idx in range(len(cam_files)):
  cam_img = str(cam_files[idx])
  img_id = str(cam_img.split("\\")[-1]).split(".")[0]
  ace_data['cam_results'][img_id] = cam_img

conv_cam_files = list(cam_result_conv_dir.glob("*.png"))
for idx in range(len(conv_cam_files)):
  cam_img = str(conv_cam_files[idx])
  img_id = str(cam_img.split("\\")[-1]).split(".")[0]
  ace_data['cam_results_conv'][img_id] = cam_img

In [223]:
masked_concept_names = [name for name in os.listdir(masked_concepts_dir) if os.path.isdir(os.path.join(masked_concepts_dir, name))]
masked_concept_names.remove("full_tcav_masked_results")
#masked_value = list(masked_concepts_dir.glob("*.json"))

'''if len(masked_value) != len(masked_concept_names):
  print("ERRORRR!!")
else:
  print("len(maksed_value) and len(masked_concept_names) are same")'''

for i in range(len(masked_concept_names)):
  name = str(masked_concept_names[i])
  concept_name = (name.split("_"))[-2]
  #mask_val = str(masked_value[i])
  '''if name not in mask_val:
    print("errror!!!")
    break'''
  each_file_dir = pathlib.Path(masked_concepts_dir, name)
  file_lists = list(each_file_dir.glob("*.png"))
  files = []
  for file in file_lists:
    files.append(str(file))
  ace_data['masked'][concept_name] = dict()
  ace_data['masked'][concept_name]['file'] = files
  #ace_data['masked'][concept_name]['masked_json'] = mask_val

result_tcav_png = list(full_masked_dir.glob("*_tcav.png"))
tcav_files = []
for file in result_tcav_png:
  tcav_files.append(str(file))

ace_data['full_masked'] = tcav_files

In [224]:
'''bn = list(tcav_data.keys())[0]
tcav_concepts = list(tcav_data[bn].keys())
len_concepts = list(ace_data['concept'].keys())

for idx in range(len(len_concepts)):
  con = tcav_concepts[idx]
  concept_data =  tcav_data[bn][con]
  avg_raw_tcav = concept_data["avg_raw_tcav"]
  for jdx in range(len(len_concepts)):
    if con+"_" in len_concepts[jdx]:
      #print(con)
      #print(len_concepts[jdx])
      ace_data['concept'][len_concepts[jdx]]["tcav_score"] = avg_raw_tcav'''


'bn = list(tcav_data.keys())[0]\ntcav_concepts = list(tcav_data[bn].keys())\nlen_concepts = list(ace_data[\'concept\'].keys())\n\nfor idx in range(len(len_concepts)):\n  con = tcav_concepts[idx]\n  concept_data =  tcav_data[bn][con]\n  avg_raw_tcav = concept_data["avg_raw_tcav"]\n  for jdx in range(len(len_concepts)):\n    if con+"_" in len_concepts[jdx]:\n      #print(con)\n      #print(len_concepts[jdx])\n      ace_data[\'concept\'][len_concepts[jdx]]["tcav_score"] = avg_raw_tcav'

In [225]:
result_png = list(results_dir.glob("*.png"))
ace_data['results']['result_image'] = str(result_png[0])
print(result_png[0])

#raw_img_val = list(results_dir.glob("*.json"))
#ace_data['results']['raw_image'] = str(raw_img_val[0])

summary_txt = list(summary_dir.glob("*.txt"))
with open(str(summary_txt[0])) as file:
  lines = file.readlines()
ace_data['summary'] = lines

summary_json = list(summary_dir.glob("*.json"))
with open(str(summary_json[0])) as f:
  filename_dict = json.load(f)
ace_data['filename_match'] = filename_dict

Ace-results2\ACE_DenseNet_airfield\results\conv5_block16_concat.png


In [226]:
with open("ace_json_results2/ace_"+model+"_"+target_class+".json", "w") as outfile:
  json.dump(ace_data, outfile, indent = 4)